In [1]:
import json

import nltk
from nltk.tokenize import sent_tokenize

from rouge_score import rouge_scorer

In [2]:
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Load dataset

In [3]:
def load_dataset(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}")
                print(f"Error: {e}")
    return data

In [4]:
train_pth = 'data/train_data.jsonl'
val_pth = 'data/val_data.jsonl'
test_pth = 'data/test_data.jsonl'

train_data = load_dataset(train_pth)
val_data = load_dataset(val_pth)
test_data = load_dataset(test_pth)

In [5]:
train_data[:2]

[{'id': 'features-41015786',
  'url': 'https://www.bbc.com/ukrainian/features-41015786',
  'title': 'Стінопис: від Філадельфії до Рабата',
  'summary': 'Вже кілька років на київських багатоповерхівках з\'являються сюжети з неприборканої уяви майстрів графіті з усього світу. Лише цього серпня принаймні на три прикрашені живописом стіни в столиці стало більше: "Просте щастя" - обличчя усміхненого чорношкірого хлопченяти, меморіальний портрет виконавця Linkin Park Честера Беннінгтона і філософська фреска "Розум, тіло і душа".',
  'text': 'Якщо для Києва "наскельне мистецтво" явище відносно нове, то на Заході - майже класичне. От лише деякі з графіті, які потрапили в фотооб\'єктив мандрівника Андрія Кондратьєва. За великим рахунком, зображення у печері Ласко у Франції, яким, як вважають вчені, приблизно 17 тисяч років, - теж графіті. Але сучасна версія стінопису зародилася в Сполучених Штатах - у Нью-Йорку і Філадельфії. І в останній еволюціонувала і стала частиною міського ДНК. Там скупче

# Baseline summarization

In [6]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text['text']) for text in dataset]
    references = [item['summary'] for item in dataset]

    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for prediction, reference in zip(summaries, references, strict=True):
        score = metric.score(prediction, reference)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    result = {metric: round((sum(values) / len(values)) * 100, 4) for metric, values in scores.items()}
    
    return result

In [7]:
print(three_sentence_summary(test_data[0]["text"]))

Виступаючи у Верховній Раді, Микола Поліщук сказав, що майже 80% з них - молоді люди віком до 30-ти років.
Він також сказав, що найбільш поширеним вживання наркотиків є у Дніпропетровській області та інших південносхідних регіонах.
Заява міністра з'явилася на тлі громадських акцій у Донецьку, де лунають скарги на легкість придбання препарату трамадол.


In [11]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

results = evaluate_baseline(val_data, scorer)

for rouge_metric, rouge_value in results.items():
    print(f"{rouge_metric.capitalize()}: {round(rouge_value, 2)}%")

Rouge1: 12.64%
Rouge2: 2.62%
Rougel: 9.16%


In [8]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

results = evaluate_baseline(test_data, scorer)

for rouge_metric, rouge_value in results.items():
    print(f"{rouge_metric.capitalize()}: {round(rouge_value, 2)}%")

Rouge1: 12.61%
Rouge2: 2.6%
Rougel: 9.15%
